<a href="https://colab.research.google.com/github/Scalable-DNN/pretrained-models/blob/main/cifar10_mobilenetV2alpha35.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!pip install larq

In [3]:
import os, sys
from pathlib import Path

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import applications as kapp, models
from tensorflow.keras.datasets import cifar10
import larq as lq

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

%matplotlib inline

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.1)

In [6]:
x_test  = tf.image.resize(x_test, (96, 96), 'bicubic').numpy()
x_train = tf.image.resize(x_train, (96, 96), 'bicubic').numpy()
x_val = tf.image.resize(x_val, (96, 96), 'bicubic').numpy()

In [7]:
print("X_train.shape: {}\tY_train.shape: {}".format(x_train.shape, y_train.shape))
print("X_val.shape:   {}\tY_val.shape:   {}".format(x_val.shape, y_val.shape))
print("X_test.shape:  {}\tY_test.shape:  {}".format(x_test.shape, y_test.shape))

X_train.shape: (45000, 96, 96, 3)	Y_train.shape: (45000, 1)
X_val.shape:   (5000, 96, 96, 3)	Y_val.shape:   (5000, 1)
X_test.shape:  (10000, 96, 96, 3)	Y_test.shape:  (10000, 1)


In [8]:
to_categorical = keras.utils.to_categorical

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [9]:
DataGen = keras.preprocessing.image.ImageDataGenerator

transformations = {
    'rotation_range': 40,
    # 'featurewise_std_normalization': True,
    # 'featurewise_center': True,
    'zoom_range': .5,
    'horizontal_flip': True,
    # 'zca_whitening': True, 
    'width_shift_range': 10 / 32, 
    'height_shift_range': 10 /32, 
    # 'brightness_range': (0, .2), 
    'rescale': 1/255.,
    'shear_range': .5,
    # 'preprocessing_function': kapp.mobilenet_v2.preprocess_input
}

train_generator = DataGen(**transformations)
val_generator   = DataGen(rescale=1/255., rotation_range=50, horizontal_flip=True, zoom_range=.3,
)
test_generator  = DataGen(rescale=1/255.)

In [10]:
# train_generator.fit(x_train)
# val_generator.fit(x_val)
# test_generator.fit(x_test)

In [11]:
# core_model = kapp.MobileNetV2()
# core_model.summary()

In [12]:
input_shape = (96, 96, 3)
fine_tune = True
# core_model = kapp.EfficientNetB0(x_train.shape[1:], weights='imagenet', include_top=False, classes=y_train.shape[-1])
core_model = kapp.MobileNetV2(include_top=False, input_shape=input_shape, classes=y_train.shape[-1], weights='imagenet', alpha=0.35)
core_model.trainable = fine_tune

inputs = keras.layers.Input(shape=input_shape)

# # x = core_model.output
# x = core_model(x, training=False)
x = core_model(inputs, training=fine_tune)
x = keras.layers.Dropout(.3)(x)
x = keras.layers.GlobalAveragePooling2D(name='global_average_pooling')(x)
# x = keras.layers.Dense(512, activation='relu')(x)
# x = keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4))(x)
# x = keras.layers.Dropout(.35)(x)
output = keras.layers.Dense(y_train.shape[-1], name='predictions', activation='softmax')(x)
# x = keras.layers.Reshape((1, 1, -1))(x)
# x = keras.layers.Dropout(.2, name='dropout', )(x)
# x = keras.layers.Conv2D(filters = y_train.shape[-1], kernel_size=1, name='logits')(x)
# x = keras.layers.Flatten()(x)
# output = keras.layers.Activation('softmax', name='predictions')(x)

# model = models.Model(inputs=core_model.input, outputs=output)
model = models.Model(inputs=inputs, outputs=output)

model.summary()
print(len(model.layers))
# lq.models.summary(model)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
mobilenetv2_0.35_96 (Functio (None, 3, 3, 1280)        410208    
_________________________________________________________________
dropout (Dropout)            (None, 3, 3, 1280)        0         
_________________________________________________________________
global_average_pooling (Glob (None, 1280)              0         
_________________________________________________________________
predictions (Dense)          (None, 10)                12810     
Total params: 423,018
Trainable params: 408,938
Non-trainable params: 14,080
_________________________________________________________________
5


In [13]:
# for i, l in enumerate(core_model.layers):
#   print(f"[{i}]   {l}")

In [14]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(core_model.layers))
core_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 142
# fine_tune_at = 106
# fine_tune_at = 80
# fine_tune_at = 0
print(type(core_model.layers[fine_tune_at]))

# Freeze all the layers before the `fine_tune_at` layer
for layer in core_model.layers[:fine_tune_at + 1]:
  layer.trainable =  False

model.summary()

Number of layers in the base model:  154
<class 'tensorflow.python.keras.layers.merge.Add'>
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
mobilenetv2_0.35_96 (Functio (None, 3, 3, 1280)        410208    
_________________________________________________________________
dropout (Dropout)            (None, 3, 3, 1280)        0         
_________________________________________________________________
global_average_pooling (Glob (None, 1280)              0         
_________________________________________________________________
predictions (Dense)          (None, 10)                12810     
Total params: 423,018
Trainable params: 219,770
Non-trainable params: 203,248
_________________________________________________________________


In [15]:
model_path = Path('/content/drive/MyDrive/0Works/cooparetive/MobilenetV2alpha35')
batch_size = 2048
epochs = 1000
initial_learning_rate = 0.008
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,
#     decay_steps=100000,
#     decay_rate=0.96,
#     staircase=True)

optimzer = keras.optimizers.Adam(initial_learning_rate)
callbacks = [
              keras.callbacks.EarlyStopping('val_accuracy', patience=10, verbose=1, restore_best_weights=True),
              keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=.5, patience=6, verbose=1),
              keras.callbacks.ModelCheckpoint(str(model_path), monitor='val_accuracy', save_best_only=True)
]

## Transfer Learning

In [16]:
# model_2 = keras.models.load_model('/content/drive/MyDrive/0Works/cooparetive/MobilenetV2alpha1_2')
model = keras.models.load_model(model_path)

OSError: ignored

In [17]:
model.compile(optimizer=optimzer, loss='categorical_crossentropy', metrics=['accuracy'], )

In [18]:
model.fit(
    train_generator.flow(x_train, y_train, batch_size),
    epochs=epochs,
    shuffle=True,
    steps_per_epoch=x_train.shape[0] // batch_size,
    validation_data=val_generator.flow(x_val, y_val, batch_size),
    validation_steps=x_val.shape[0] // batch_size,
    callbacks=callbacks,
)

Epoch 1/1000


KeyboardInterrupt: ignored

In [19]:
model.evaluate(test_generator.flow(x_test, y_test, batch_size=batch_size))

2/5 [===========>..................] - ETA: 17s - loss: 2.7421 - accuracy: 0.1069

KeyboardInterrupt: ignored

In [ ]:
model.save('mobilnetV2alpha35')

INFO:tensorflow:Assets written to: mobilnetV2alpha1/assets


INFO:tensorflow:Assets written to: mobilnetV2alpha1/assets


In [ ]:
!tar -czvf 'mobilnetV2alpha35.tar.gz' 'mobilnetV2alpha35'